In [3]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
data.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [5]:
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [6]:
y = data.pop('Survived')

In [9]:
import tensorflow as tf
import numpy as np

Fare전처리레이어 = tf.keras.layers.Normalization(axis=None)
Fare전처리레이어.adapt(np.array(data['Fare']))
# print(Fare전처리레이어(np.array(data['Fare'])))

SibSp전처리레이어 = tf.keras.layers.Normalization(axis=None)
SibSp전처리레이어.adapt(np.array(data['SibSp']))

Parch전처리레이어 = tf.keras.layers.Normalization(axis=None)
Parch전처리레이어.adapt(np.array(data['Parch']))

Pclass전처리레이어 = tf.keras.layers.Normalization(axis=None)
Pclass전처리레이어.adapt(np.array(data['Pclass']))

Age전처리레이어 = tf.keras.layers.Discretization(bin_boundaries=[10,20,30,40,50,60])


Sex전처리레이어 = tf.keras.layers.StringLookup(output_mode='one_hot')
Sex전처리레이어.adapt(np.array(data['Sex']))

Embarked전처리레이어 = tf.keras.layers.StringLookup(output_mode='one_hot')
Embarked전처리레이어.adapt(np.array(data['Embarked']))

Ticket전처리레이어 = tf.keras.layers.StringLookup()
Ticket전처리레이어.adapt(np.array(data['Ticket']))

print(len(data['Ticket'].unique()))
Ticket임베딩레이어  = tf.keras.layers.Embedding(len(data['Ticket'].unique())+1,9)


681


In [12]:
from os import name
# 1. 맨 위는 Input레이어
input_fare = tf.keras.Input(shape=(1,), name='Fare')
input_parch = tf.keras.Input(shape=(1,), name='Parch')
input_sibsp = tf.keras.Input(shape=(1,), name='SibSp')
input_pclass = tf.keras.Input(shape=(1,), name='Pclass')
input_age = tf.keras.Input(shape=(1,), name='Age')
input_sex = tf.keras.Input(shape=(1,), name='Sex', dtype=tf.string)
input_embarked= tf.keras.Input(shape=(1,), name='Embarked', dtype=tf.string)
input_ticket = tf.keras.Input(shape=(1,), name='Ticket', dtype=tf.string)

In [13]:
# 2. Input을 전처리레이어에 넣기
x_fare = Fare전처리레이어(input_fare)
x_parch = Parch전처리레이어(input_parch)
x_sibsp = SibSp전처리레이어(input_sibsp)
x_pclass = Pclass전처리레이어(input_pclass)
x_age = Age전처리레이어(input_age)
x_sex = Sex전처리레이어(input_sex)
x_embarked = Embarked전처리레이어(input_embarked)
x_ticket_onehot = Ticket전처리레이어(input_ticket)
x_ticket_embed = Ticket임베딩레이어(x_ticket_onehot)
x_ticket = tf.keras.layers.Flatten()(x_ticket_embed)

In [14]:
# 3. 레이어들 하나로 합치기
concat1 = tf.keras.layers.concatenate([
  x_fare, x_parch, x_sibsp, x_pclass, x_age, x_sex, x_embarked, x_ticket
])

x = tf.keras.layers.Dense(128, activation='relu')(concat1)
x = tf.keras.layers.Dense(64, activation='relu')(x)
마지막레이어 = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [15]:
# 4, 모델의 시작과 끝 레이어 알려주기
model = tf.keras.Model(
    inputs=[input_fare, input_parch, input_sibsp, input_pclass, input_age, input_sex, input_embarked, input_ticket],
    outputs=마지막레이어
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Ticket (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ string_lookup_2     │ (None, 1)         │          0 │ Ticket[0][0]      │
│ (StringLookup)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Fare (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Parch (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ SibSp (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Pclass (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Age (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Sex (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Embarked            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 9)      │      6,138 │ string_lookup_2[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 1)         │          3 │ Fare[0][0]        │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_6     │ (None, 1)         │          3 │ Parch[0][0]       │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_5     │ (None, 1)         │          3 │ SibSp[0][0]       │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_7     │ (None, 1)         │          3 │ Pclass[0][0]      │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ discretization_1    │ (None, 1)         │          0 │ Age[0][0]         │
│ (Discretization)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ string_lookup       │ (None, 3)         │          0 │ Sex[0][0]         │
│ (StringLookup)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ string_lookup_1     │ (None, 4)         │          0 │ Embarked[0][0]    │
│ (StringLookup)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 9)         │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 21)        │          0 │ normalization_4[… │
│ (Concatenate)       │                   │            │ normalization_6[… │
│                     │                   │            │ normalization_5[

 Total params: 17,287 (67.54 KB)

 Trainable params: 17,275 (67.48 KB)

 Non-trainable params: 12 (64.00 B)

In [16]:
x데이터 = {
    'Fare': np.array(data['Fare']),
    'Parch': np.array(data['Parch']),
    'SibSp': np.array(data['SibSp']),
    'Pclass': np.array(data['Pclass']),
    'Age': np.array(data['Age']),
    'Sex': np.array(data['Sex']),
    'Embarked': np.array(data['Embarked']),
    'Ticket': np.array(data['Ticket'])
}
model.fit(x데이터, np.array(y), epochs=15, validation_split=0.1)

Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6059 - loss: 0.6571 - val_accuracy: 0.7889 - val_loss: 0.4927
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7616 - loss: 0.5050 - val_accuracy: 0.8111 - val_loss: 0.4172
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8019 - loss: 0.4547 - val_accuracy: 0.8333 - val_loss: 0.3946
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8041 - loss: 0.4301 - val_accuracy: 0.8444 - val_loss: 0.3870
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8479 - loss: 0.3817 - val_accuracy: 0.8444 - val_loss: 0.3653
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8850 - loss: 0.3147 - val_accuracy: 0.8333 - val_loss: 0.3476
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9014 - loss: 0.3035 - val_accuracy: 0.8222 - val_loss: 0.3596
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9420 - loss: 0.1770 - val_accuracy: 0.8222 - val_loss